# 🕵️ Phase 1 : Analyse Mitre & Compréhension

**Objectifs** :
1. Comprendre les attaques présentes dans le dataset.
2. Mapper ces attaques avec le framework **MITRE ATT&CK**.
3. Analyser les caractéristiques réseaux (Ports, Flags) de chaque attaque.

---

## 1. Mapping MITRE ATT&CK

Le framework MITRE ATT&CK est la base de connaissances mondiale des tactiques adverses.
Voici comment nos attaques CICIDS2017 se classent :

| Attaque CICIDS2017 | Tactique MITRE | Technique ID | Description |
|--------------------|----------------|--------------|-------------|
| **Brute Force (FTP/SSH)** | Credential Access | T1110 | Essayer plein de mots de passe. |
| **DoS / DDoS** | Impact | T1498 | Saturer le réseau pour le rendre indisponible. |
| **Port Scanning** | Discovery | T1046 | Chercher des portes ouvertes sur le réseau. |
| **Web Attacks (SQL/XSS)** | Initial Access / Exfiltration | T1190 | Exploiter une faille dans une application web. |
| **Botnet** | Command and Control | T1043 | Machines zombies contrôlées à distance. |

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Charger le dataset (échantillon ou complet)
df = pd.read_csv('../../data/processed/cicids2017_cleaned.csv')

# Focus sur les attaques
attacks = df[df['Attack Type'] != 'Normal Traffic']

## 2. Analyse des Ports (Discovery)

Certaines attaques visent des ports spécifiques (SSH=22, HTTP=80). Vérifions cela.

In [ ]:
# Top Ports ciblés par type d'attaque
top_ports = attacks.groupby(['Attack Type', 'Destination Port']).size().reset_index(name='count')
top_ports = top_ports.sort_values(['Attack Type', 'count'], ascending=[True, False]).groupby('Attack Type').head(3)

print("Top 3 Ports ciblés par attaque :")
print(top_ports)

**Analyse attendue** :
- **Web Attacks** devraient cibler les ports 80 (HTTP) ou 443 (HTTPS).
- **SSH-Patator** devrait cibler le port 22.
- **FTP-Patator** devrait cibler le port 21.

## 3. Analyse Temporelle (Comportement)

Comment se comporte la durée des flux (`Flow Duration`) selon l'attaque ?

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='Attack Type', y='Flow Duration', data=attacks)
plt.yscale('log') # Log scale car les écarts sont énormes
plt.xticks(rotation=45)
plt.title('Distribution de la durée des flux par attaque (Log Scale)')
plt.show()

**Analyse** :
- Un **DoS** peut avoir des durées très longues (saturer la connexion) ou très courtes (mitraillette).
- Un **Brute Force** est souvent une succession de connexions courtes.